In [1]:
import json

In [2]:
with open("documents.json", "rt") as f_in:
    docs_raw = json.load(f_in)

In [3]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [4]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [5]:
len(documents)

948

In [6]:
from minsearch import Index

index = Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [7]:
index.fit(documents)

In [8]:
q = "the course is already started, can I still enroll?"
boost = {
    'question': 3.0,
    'section': 0.5
}

results = index.search(
    query=q,
    boost_dict=boost,
    num_results=5, 
    filter_dict={'course': 'data-engineering-zoomcamp'}
)

In [9]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [10]:
from openai import OpenAI

In [11]:
client = OpenAI()

In [12]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering the QUESTION. 
If the CONTEXT doesn't contain the ansewer, output NONE.

QUESTION: {question}

CONTEXT: {context}
"""

context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [13]:
prompt = prompt_template.format(question=q, context=context).strip()

In [14]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering the QUESTION. 
If the CONTEXT doesn't contain the ansewer, output NONE.

QUESTION: the course is already started, can I still enroll?

CONTEXT: section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your 

In [15]:
response = client.chat.completions.create(
    model='gpt-4o', 
    messages=[{'role': "user", 'content': prompt}]
)

response.choices[0].message.content

"Yes, you can still enroll in the course after it has started. Even if you haven't registered, you're still eligible to submit the homeworks. However, be aware of the deadlines for turning in the final projects."

In [16]:
def search(query, course):
    boost = {
        'question': 3.0,
        'section': 0.5
    }

    results = index.search(
        query=query,
        boost_dict=boost,
        num_results=5, 
        filter_dict={'course': course}
    )

    return results

In [17]:
def build_prompt(query, results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
    Use only the facts from the CONTEXT when answering the QUESTION. 
    If the CONTEXT doesn't contain the ansewer, output NONE.

    QUESTION: {question}

    CONTEXT: {context}
    """

    context = ""

    for doc in results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [18]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o', 
        messages=[{'role': "user", 'content': prompt}]
    )

    return response.choices[0].message.content

In [23]:
query = 'how do I run kafka?'
results = search(query, course='data-engineering-zoomcamp')
prompt = build_prompt(query, results)

In [24]:
llm(prompt)

'To run Kafka using Java, in the project directory, execute the following command in the terminal:\n\n```sh\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\nReplace `<jar_name>` with the name of your jar file.'

In [25]:
def rag(query):
    search_results = search(query, course='data-engineering-zoomcamp')
    prompt = build_prompt(query, search_results)
    return llm(prompt)

In [26]:
rag(query="the course has already started, can I still join?")

"Yes, even if the course has already started, you can still join and are eligible to submit the homeworks. Be aware that there will be deadlines for turning in the final projects, so don't leave everything for the last minute."

In [27]:
from elasticsearch import Elasticsearch

In [28]:
es_client = Elasticsearch('http://localhost:9200')

In [29]:
es_client.info()

{'name': 'b400d59e3fa3',
 'cluster_name': 'docker-cluster',
 'cluster_uuid': 'aoGHGEnYQFOXpxGFr1Jcgw',
 'version': {'number': '8.4.3',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73',
  'build_date': '2022-10-04T07:17:24.662462378Z',
  'build_snapshot': False,
  'lucene_version': '9.3.0',
  'minimum_wire_compatibility_version': '7.17.0',
  'minimum_index_compatibility_version': '7.0.0'},
 'tagline': 'You Know, for Search'}

In [30]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

{'acknowledged': True,
 'shards_acknowledged': True,
 'index': 'course-questions'}

In [32]:
from tqdm.auto import tqdm

In [35]:
for doc in tqdm(documents):
    es_client.index(index=index_name, body=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [36]:
query = "the course already started, can I still join?"

In [44]:
def search_es(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    result_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [45]:
search_es(query)

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at

In [46]:
def rag(query):
    search_results = search_es(query)
    prompt = build_prompt(query, search_results)
    return llm(prompt)

In [47]:
rag(query)

"Yes, you can still join the course after it has started. Even if you don't register, you're eligible to submit the homeworks. However, keep in mind that there will be deadlines for the final projects, so it's important not to leave everything for the last minute."